In [7]:
from lib.utils import generate_mean_ensemble_metrics, aggregate_pred_dataframe, generate_mean_ensemble_metrics_auto
import os
import pandas as pd
test_csv_fliles = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'test' in file]
test_csv_fliles.sort()
test_csv_fliles = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in test_csv_fliles]

valid_csv_fliles = [file for file in os.listdir('result_cv/esm-33-gearnet-ensemble/fold_0/intermediate') if 'valid' in file]
valid_csv_fliles.sort()
valid_csv_fliles = [f'result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/{file}' for file in valid_csv_fliles]


In [8]:
agg = pd.DataFrame()
for csv_file in test_csv_fliles:
    df = aggregate_pred_dataframe([csv_file])
    me = generate_mean_ensemble_metrics(df)
    print(f'Processing {csv_file}, aggregated metrics: {me["mcc"]})')
    agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)

agg.aggregate(['mean', 'std'])


Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_0_test.csv, aggregated metrics: 0.6165)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_10_test.csv, aggregated metrics: 0.6517)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_11_test.csv, aggregated metrics: 0.6441)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_12_test.csv, aggregated metrics: 0.6438)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_13_test.csv, aggregated metrics: 0.6408)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_14_test.csv, aggregated metrics: 0.6682)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_15_test.csv, aggregated metrics: 0.6073)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_16_test.csv, aggregated metrics: 0.6186)
Processing result_cv/esm-33-gearnet-ensemble/fold_0/intermediate/iter_17_test.csv, aggregated metrics: 0.6609)
Pr

,sensitivity,specificity,accuracy,precision,mcc,micro_auroc
mean,0.574770,0.990440,0.968906,0.769696,0.648898,0.919362
std,0.037646,0.002288,0.001143,0.033572,0.014627,0.010408


In [9]:
# for valid_csv, test_csv in zip(valid_csv_fliles, test_csv_fliles):
#     valid_df = aggregate_pred_dataframe([valid_csv])
#     test_df = aggregate_pred_dataframe([test_csv])
#     me = generate_mean_ensemble_metrics_auto(valid_df, test_df)
#     agg = pd.concat([agg, pd.DataFrame([me])], ignore_index=True)


# agg.aggregate(['mean', 'std'])

In [10]:
import pandas as pd

def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"

# Sample loading of your CSV - this would be replaced with your actual loading
result_df = pd.read_csv('result_cv/result_cv.csv')

# metrics = ['mcc', 'sensitivity', 'specificity', 'accuracy', 'precision', 'micro_auroc']
metrics = ['mcc', 'sensitivity', 'precision', 'micro_auroc']
grouped = result_df.groupby('model_key').agg({
    k: ['mean', 'std'] for k in metrics
})

record_counts = result_df.groupby('model_key').size()

for metric in metrics:
    grouped[f'{metric}_formatted'] = grouped.apply(lambda row: format_mean_std(row, metric), axis=1)


# Add the record count column to the grouped dataframe
grouped['record_count'] = record_counts.values

formatted_df = grouped[[k + '_formatted' for k in metrics] + ['record_count']]
formatted_df.columns = metrics + ['record_count']
formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

formatted_df = formatted_df.reset_index()
formatted_df

,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,gearnet,0.453 ± 0.023,0.392 ± 0.053,0.584 ± 0.063,0.860 ± 0.011,5
1,bert,0.499 ± 0.010,0.432 ± 0.033,0.630 ± 0.051,0.892 ± 0.012,5
2,bert-gearnet,0.519 ± 0.031,0.521 ± 0.033,0.568 ± 0.062,0.905 ± 0.016,5
3,bert-gearnet-ensemble,0.578 ± 0.018,0.517 ± 0.041,0.691 ± 0.021,0.913 ± 0.006,5
4,esm-33-gearnet-ensemble-rus,0.656 ± 0.019,0.747 ± 0.016,0.612 ± 0.044,0.945 ± 0.004,5
5,esm-t33,0.663 ± 0.020,0.661 ± 0.009,0.701 ± 0.035,0.947 ± 0.010,5
6,esm-33-gearnet,0.667 ± 0.013,0.645 ± 0.028,0.725 ± 0.025,0.935 ± 0.013,5
7,esm-t33-ensemble,0.684 ± 0.002,0.672 ± 0.016,0.729 ± 0.018,0.952 ± 0.003,5
8,esm-33-gearnet-ensemble,0.686 ± 0.005,0.658 ± 0.036,0.749 ± 0.030,0.951 ± 0.003,5
9,esm-33-gearnet-resiboost-n25,0.690 ± 0.009,0.666 ± 0.043,0.749 ± 0.033,0.931 ± 0.006,5


In [11]:
from tabulate import tabulate

def visualize(formatted_df, metrics):
    markdown_table = tabulate(formatted_df[['model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
    print(markdown_table)

visualize(formatted_df=formatted_df, metrics=metrics)

| model_key                    | mcc           | sensitivity   | precision     | micro_auroc   |
|:-----------------------------|:--------------|:--------------|:--------------|:--------------|
| gearnet                      | 0.453 ± 0.023 | 0.392 ± 0.053 | 0.584 ± 0.063 | 0.860 ± 0.011 |
| bert                         | 0.499 ± 0.010 | 0.432 ± 0.033 | 0.630 ± 0.051 | 0.892 ± 0.012 |
| bert-gearnet                 | 0.519 ± 0.031 | 0.521 ± 0.033 | 0.568 ± 0.062 | 0.905 ± 0.016 |
| bert-gearnet-ensemble        | 0.578 ± 0.018 | 0.517 ± 0.041 | 0.691 ± 0.021 | 0.913 ± 0.006 |
| esm-33-gearnet-ensemble-rus  | 0.656 ± 0.019 | 0.747 ± 0.016 | 0.612 ± 0.044 | 0.945 ± 0.004 |
| esm-t33                      | 0.663 ± 0.020 | 0.661 ± 0.009 | 0.701 ± 0.035 | 0.947 ± 0.010 |
| esm-33-gearnet               | 0.667 ± 0.013 | 0.645 ± 0.028 | 0.725 ± 0.025 | 0.935 ± 0.013 |
| esm-t33-ensemble             | 0.684 ± 0.002 | 0.672 ± 0.016 | 0.729 ± 0.018 | 0.952 ± 0.003 |
| esm-33-gearnet-ensemble     

In [12]:
target_df = formatted_df[formatted_df['model_key'].str.match('esm-33-gearnet-\d+') | formatted_df['model_key'].str.match('esm-t33-\d+')]

visualize(formatted_df=target_df, metrics=metrics)

| model_key   | mcc   | sensitivity   | precision   | micro_auroc   |
|-------------|-------|---------------|-------------|---------------|
